# Run a training script with the Python SDK

You can use the Python SDK for Azure Machine Learning to submit scripts as jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **diabetes_training.py** script that can be found in the **diabetes-training** folder. It uses the **diabetes.csv** file as the training data. The **environment.yml** file lists the necessary packages which need to be installed on the compute cluster before running the script.

**Important**: Replace the **XX** in `vm-cluster-XX` with your initials. The name should correspond with the name of an existing compute cluster in your Azure Machine Learning workspace.

In [3]:
# import necessary packages
from azure.ai.ml import MLClient, command, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment
from azureml.core import Workspace

if __name__ == "__main__":
    # get details of the current Azure ML workspace
    ws = Workspace.from_config()

    # default authentication flow for Azure applications
    default_azure_credential = DefaultAzureCredential()
    subscription_id = ws.subscription_id
    resource_group = ws.resource_group
    workspace = ws.name

    # client class to interact with Azure ML services and resources, e.g. workspaces, jobs, models and so on.
    ml_client = MLClient(
        default_azure_credential,
        subscription_id,
        resource_group,
        workspace)

    env_name = "sklearn-env"
    env_docker_image = Environment(
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
        name=env_name,
        conda_file="environment.yml",
    )
    ml_client.environments.create_or_update(env_docker_image)

    # configure job
    computeName="aml-cluster"
    job = command(
        code="./diabetes-training",
        command="python diabetes_training.py",
        environment=f"{env_name}@latest",
        compute=computeName,
        display_name="diabetes-pythonv2-train",
        experiment_name="diabetes-training"
    )

    # submit job
    returned_job = ml_client.create_or_update(job)
    aml_url = returned_job.studio_url
    print("Monitor your job at", aml_url)